<a href="https://colab.research.google.com/github/GianlucaRapaglia/LLM-training/blob/main/01%20-%20LLM%20validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>